In [1]:
import pandas as pd
import numpy as np 
import pylab as P 
# Import the module for plotting
import matplotlib.pyplot as plt
from pandas import Series,DataFrame,merge

In [2]:
# data import
dailyData = pd.read_csv('/Users/jessia/psychSignalData/data/stocktwits_twitter_withretweets_daily.csv')

In [3]:
# symbols import
symbStocks = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/stocks.csv')
# section symbols
secMat = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/basic_materials.csv')
secCong = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/conglomerates.csv')
secConsu = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/consumer_goods.csv')
secFinan = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/financial.csv')
secHealth = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/healthcare.csv')
secIndus = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/industrial_goods.csv')
secServ = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/services.csv')
secTech = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/technology.csv')
secUtil = pd.read_csv('/Users/jessia/psychSignalData/data/symbols/utilities.csv')


In [4]:
symbStocks.dtypes
symbStocks = symbStocks.ix[:,0:3]

In [5]:
symbStocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9060 entries, 0 to 9059
Data columns (total 3 columns):
EXCHANGE    9060 non-null object
 SYMBOL     9060 non-null object
 TITLE      9060 non-null object
dtypes: object(3)
memory usage: 283.1+ KB


In [ ]:
# type of sector data
print(secMat.dtypes)
print(secCong.dtypes)
print(secConsu.dtypes)
print(secFinan.dtypes)
print(secHealth.dtypes)
print(secIndus.dtypes)
print(secServ.dtypes)
print(secTech.dtypes)
print(secUtil.dtypes)

In [ ]:
dailyData.dtypes

In [ ]:
# information for data set
# daily data rows = 7,120,506
dailyData.info() 

In [7]:
dailyData.head()

,SOURCE,SYMBOL,TIMESTAMP_UTC,BULLISH_INTENSITY,BEARISH_INTENSITY,BULL_MINUS_BEAR,BULL_SCORED_MESSAGES,BEAR_SCORED_MESSAGES,BULL_BEAR_MSG_RATIO,TOTAL_SCANNED_MESSAGES
0,stocktwits+twitter_withretweets,10GEN,2011-04-08T04:00:00Z,0,0,0,0,0,0,2
1,stocktwits+twitter_withretweets,10GEN,2011-04-16T04:00:00Z,0,0,0,0,0,0,3
2,stocktwits+twitter_withretweets,10GEN,2011-04-17T04:00:00Z,0,0,0,0,0,0,2
3,stocktwits+twitter_withretweets,10GEN,2011-04-20T04:00:00Z,0,0,0,0,0,0,1
4,stocktwits+twitter_withretweets,10GEN,2011-07-19T04:00:00Z,0,0,0,0,0,0,3


In [ ]:
dailyData.describe()

In [16]:
# PLOT A SAMPLE: TOTAL SCANNED MESSAGES BETWEEN 1 TO 11
dailyData['TOTAL_SCANNED_MESSAGES'].hist(range=(1,10))
P.show()

In [ ]:
# plt.show(dailyData.plot(kind = 'box'))

In [17]:
# merge dataset left join
dailyStockSymb = merge(dailyData,symbStocks,left_on='SYMBOL',right_on = ' SYMBOL',how = "left")

In [18]:
dailyStockSymb = dailyStockSymb.ix[:,1:11]
dailyStockSymb.columns

Index(['SYMBOL', 'TIMESTAMP_UTC', 'BULLISH_INTENSITY', 'BEARISH_INTENSITY',
       'BULL_MINUS_BEAR', 'BULL_SCORED_MESSAGES', 'BEAR_SCORED_MESSAGES',
       'BULL_BEAR_MSG_RATIO', 'TOTAL_SCANNED_MESSAGES', 'EXCHANGE'],
      dtype='object')

In [27]:
# Group By Stock Exchange
exchangeGrouped = dailyStockSymb.groupby(dailyStockSymb['EXCHANGE'])
sumByExchange = exchangeGrouped.sum()
print(sumByExchange.columns)
# sumByExchange

Index(['BULLISH_INTENSITY', 'BEARISH_INTENSITY', 'BULL_MINUS_BEAR',
       'BULL_SCORED_MESSAGES', 'BEAR_SCORED_MESSAGES', 'BULL_BEAR_MSG_RATIO',
       'TOTAL_SCANNED_MESSAGES'],
      dtype='object')


In [29]:
# statistic description for bull bear messages ratio by exchanges
statByExchange_Ratio = exchangeGrouped['BULL_BEAR_MSG_RATIO'].describe()
statByExchange = exchangeGrouped.describe()
# export the statistic description by exchanges
statByExchange.to_csv('/Users/jessia/psychSignalData/data/statByExchange.csv', encoding='utf-8')

In [ ]:
# a table only contains the total quality of bullish, bearish and total messages for each exchange
messagesTable = sumByExchange.ix[:,[3,4,6]]
messagesTable

In [ ]:
# the percentage of bull messages 
bullPerc = messagesTable['BULL_SCORED_MESSAGES']/messagesTable['TOTAL_SCANNED_MESSAGES']
# the percentage of bear messages
bearPerc = messagesTable['BEAR_SCORED_MESSAGES']/messagesTable['TOTAL_SCANNED_MESSAGES']

In [ ]:
# neutral = total - bull - bear
neutralCounts = messagesTable['TOTAL_SCANNED_MESSAGES']-messagesTable['BULL_SCORED_MESSAGES']-messagesTable['BEAR_SCORED_MESSAGES']
# print (neutralCounts)
neutralPerc = neutralCounts/messagesTable['TOTAL_SCANNED_MESSAGES']

In [ ]:
# check the total percentage = 1 or not
totalPerc = bullPerc + bearPerc + neutralPerc
# totalPerc

In [ ]:
# add new columns to the table. by using 'insert(idx, col_name, value)'
messagesTable.insert(2, 'NEUTRAL_SCORED_MESSAGES', neutralCounts)
# messagesTable

In [ ]:
# change data format to %
bullPerc = bullPerc.apply(lambda x: format(x, '.2%'))
bearPerc = bearPerc.apply(lambda x: format(x, '.2%'))
neutralPerc = neutralPerc.apply(lambda x: format(x, '.2%'))

In [ ]:
# sentiment summary by stock exchange
sentiOutcome = messagesTable
sentiOutcome.insert(1,'BULL_PERC', bullPerc)
sentiOutcome.insert(3,'BEAR_PERC', bearPerc)
sentiOutcome.insert(5,'NEUTRAL_PERC', neutralPerc)
# sentiOutcome

In [ ]:
# export to a csv file
sentiOutcome.to_csv('/Users/jessia/psychSignalData/data/sentimentSummaryByExchange.csv', encoding='utf-8')